# Technical Report
### NYC Homeless Shelter Population Forecasting

---

### Problem Statement
In this project, I forecasted the total number of individuals sheltered by the NYC Department of Homeless Services (DHS). I used DHS Daily Report publicly available from [NYC Open Data](https://opendata.cityofnewyork.us) to build Time Series model. I aimed to help understand homeless communities better by examining different factors that change the number of indivuals in DHS shelter and to help better prepare for those who are in need of a space at a shelter by forecasting the general demand.

### Data Collection
1. DHS Daily Report (Source: https://data.cityofnewyork.us/Social-Services/DHS-Daily-Report/k46n-sa2m)

DHS Daily Report was scraped using [Socrata Open Data API (SODA)](https://dev.socrata.com/foundry/data.cityofnewyork.us/k46n-sa2m). The report includes the number of total individuals in Shelter and the breakdown of the shelter population such as the total number of children in shelter from 2013. This report is updated daily and the update is availble through the API. Although the data did not have any null values, there were some missing dates indicating the report was not made on some days. For this reason, the daily data was resampled into weekly average data for analysis.

### Database (PostgreSQL)
I built PostgreSQL database on Heroku for this project. The database was contructed for the following reasons. First, the number of rows that are returned from Socrata Open Data API is limited. By contructing a database, I can keep the rows when they are no longer available through the API. Second, as I contruct database with properly cleaned data, I do not have to repeat the data cleaning step when the dataset is used in other Jupyter notebooks or different visualization tools such as Tableau or Dash.

### Exploratory Data Analysis (EDA)
The EDA for the project was largely divided into two parts. The first part was to examine the trend in DHS shelter demographic. The second part of EDA was to understand the trend and seasonality in the total individuals in shelter in order to perform time series analysis.

**Part 1. DHS Shelter Demographic**                          
While the ratio of children to adult and the ratio of people with and without family shows clear trend with a slight seasonality, the ratio of women and men exibits a slight trend and a strong seasonality.

>**Ratio of Children to Adults in DHS Shelter**

<img src="production/assets/age.png" width="500"/>
The ratio has been constant until 2015 and gradually decrased since then. There seems to be some seasonality. It is also noted that there are always more adult than children.

---

> **Ratio of people with family to people without family in DHS Shelter**

<img src="production/assets/family.png" width="500"/>

This Ratio follows similar pattern to the ratio of children and adult. This is because the children would not have been admitted into the shelter and would have been booked into different facility such as orphange if they were not with their family. Towards the end of 2014 and 2016, two picks were observed that were not observed in the ratio of children to adult plot. This indicates that there seems to have been an increase in the family with no children that needed the support from DHS. It is also noted that there are always more people with their family and people without their family.

---

> **Ratio of women to men with no family in DHS Shelter**

<img src="production/assets/gender.png" width="500"/>

- There seems to be no apparent trend in this ratio change. However, there is a strong seasonality that matches the seasonality pattern were seen in the change in the ratio of people with family to people without family and the ratio of children to adult in DHS shelter. This indicates that there is a higher demand of families and single women for a shelter space during the time the peaks are observed. It is also noted that there are always more men than women in the shelter even when there is a higher booking rate of females.

---

**Part 2. Total Individuals in DHS Shelter**
<img src="production/assets/total.png" width="500"/>

<img src="production/assets/seasonal.png" width="500"/>

- The total individuals in DHS Shelter increased rapidly from 2014 to 2015 and showed gradual increase until 2017. From 2017 until now, the upward trend has been stagnant.

- There is a sign of seasonality. Towards the end of the year through the beggining of the next year, the population in the shelter increases. This seasonality was not prominent from 2013 to 2014 and 2015 to 2016.

### Times Series Modeling          

In order to build a time series model, three terms, `p`, `d`, and `q` was defined.

`p` (AR: AutoRegression)
- The term `p` is used to have the model comprehend a long term trend in the data. The Partial Autocorrelation Function (PACF) plot below can be used to identify the term `p`. In the PACF plot below, a sharp cut-off in lag 3 was identifed. Lag 1 value was positive and, in such pattern, `p` term is the lag right before the sharpt cut-off. Therefore, **`p` was selected to be 1**

<img src="production/assets/pacf.png" width="400"/>

`d` (I: Integration)
- The term `d` is used to make the data stationary. Augmented Dickey-Fuller (ADF) Test was used to determine if the data was stationary. If the ADF test result indicated that the data was not stationary, the data was differenced until stationarity was observed. For the weekly DHS shelter population dataset, stationarity was observed when the dataset was differenced once. Therefore, **`d` was selected to be 1**.

`q` (MA: Moving Average)
- The term `q` is used to help the model understand short and sudden fluctuation in the dataset. In order to determine the term `q`, Autocorrelation Fucntion (ACF) plot can be used. In ACF plot below, no sharp cut-off was observed, which indicates that there is no sudden fluctuation in the data for the model to learn. Therefore, **`q` is selected to be 0**.

<img src="production/assets/acf.png" width="400"/>

--- 

> **ARIMA (1, 1, 0) model** 

<img src="production/assets/arima.png" width="650"/>

- The prediction converged quickly and failed to make meaningful predictions.
- Root mean square error (RMSE): 1834

---

> **SARIMA (1, 1, 0) x (2, 2, 0, 51)**

<img src="production/assets/sarima.png" width="650"/>

- Grid Search was used to find the seasonal order terms that optimizes the result.
- Using seasonal order terms significantly improved the performance of the model. 
- Root mean square error (RMSE): 560

### Data Visualization

### Next Steps